In [ ]:
# 1. RANDOM FOREST

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle


In [2]:
X_train = pd.read_csv("../../data/processed/X_train.csv")
y_train = pd.read_csv("../../data/processed/y_train.csv")["RUL_steps"].values
X_val = pd.read_csv("../../data/processed/X_val.csv")
y_val = pd.read_csv("../../data/processed/y_val.csv")["RUL_steps"].values
train_balanced =  pd.read_csv("../../data/processed/train_balanced.csv")
train_df = pd.read_csv("../../data/processed/train_df.csv")
val_df = pd.read_csv("../../data/processed/val_df.csv")

In [3]:
weights = train_balanced['risk_bin'].map({
    'MUY_CRITICO': 8.0,
    'CRITICO': 5.0, 
    'ALTO_RIESGO': 2.0,
    'BAJO_RIESGO': 1.0
}).values

# 1. RANDOM FOREST (NO necesita escalado)
rf_model = RandomForestRegressor(n_estimators=500, max_depth=10, min_samples_split=10,
                                min_samples_leaf=5, max_features='sqrt', random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train, sample_weight=weights)
rf_pred = rf_model.predict(X_val)

# MÉTRICAS RANDOM FOREST (CRÍTICOS RUL<50)
mask_critica = y_val < 50
rf_mae_total = mean_absolute_error(y_val, rf_pred)
rf_mae_criticos = mean_absolute_error(y_val[mask_critica], rf_pred[mask_critica])
rf_r2_total = r2_score(y_val, rf_pred)

print(f"📊 RANDOM FOREST:")
print(f"MAE total:     {rf_mae_total:.2f}")
print(f"MAE RUL<50:    {rf_mae_criticos:.2f}")
print(f"R² total:      {rf_r2_total:.4f}")
print(f"Nº críticos:   {mask_critica.sum():,} ({mask_critica.mean()*100:.1f}%)")
print()




📊 RANDOM FOREST:
MAE total:     35.71
MAE RUL<50:    12.23
R² total:      -0.3691
Nº críticos:   24,643 (62.1%)



In [5]:


param_dist = {
    'n_estimators': [400, 450, 500, 550, 600],
    'max_depth': [8, 10, 12, 15],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [2, 3, 5, 8],
    'max_features': ['sqrt', 0.3, 0.4]
}

rf_fast = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(
    rf_fast, param_dist,
    n_iter=60,  # 60 mejores combinaciones (45-90 min total)
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=1,
    verbose=2,
    random_state=42
)

print("🚀 RandomizedSearch iniciado: 60 iteraciones (~1 hora)")
random_search.fit(X_train, y_train, sample_weight=weights)

# Resultados completos
best_rf = random_search.best_estimator_
best_pred = best_rf.predict(X_val)

mask_critica = y_val < 50
mae_total = mean_absolute_error(y_val, best_pred)
mae_criticos = mean_absolute_error(y_val[mask_critica], best_pred[mask_critica])
rmse_total = np.sqrt(mean_squared_error(y_val, best_pred))
r2_total = r2_score(y_val, best_pred)

print("\n📊 RESULTADOS:")
print(f"MAE total: {mae_total:.2f} (original: {rf_mae_total:.2f})")
print(f"MAE RUL<50: {mae_criticos:.2f} (original: {rf_mae_criticos:.2f})")
print(f"RMSE: {rmse_total:.2f}")
print(f"R²: {r2_total:.4f}")

print("\n💾 TU MODELO FINAL:")
print("rf_model = RandomForestRegressor(")
for param, value in random_search.best_params_.items():
    print(f"    {param}={value},")
print("    random_state=42, n_jobs=-1)")


🚀 RandomizedSearch iniciado: 60 iteraciones (~1 hora)
Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV] END max_depth=10, max_features=0.4, min_samples_leaf=5, min_samples_split=15, n_estimators=400; total time= 2.4min
[CV] END max_depth=10, max_features=0.4, min_samples_leaf=5, min_samples_split=15, n_estimators=400; total time= 2.5min
[CV] END max_depth=10, max_features=0.4, min_samples_leaf=5, min_samples_split=15, n_estimators=400; total time= 2.6min
[CV] END max_depth=10, max_features=0.3, min_samples_leaf=8, min_samples_split=10, n_estimators=400; total time= 1.8min
[CV] END max_depth=10, max_features=0.3, min_samples_leaf=8, min_samples_split=10, n_estimators=400; total time= 1.8min
[CV] END max_depth=10, max_features=0.3, min_samples_leaf=8, min_samples_split=10, n_estimators=400; total time= 1.8min
[CV] END max_depth=8, max_features=sqrt, min_samples_leaf=8, min_samples_split=10, n_estimators=600; total time= 1.7min
[CV] END max_depth=8, max_features=sqrt, min

In [6]:
best_rf = RandomForestRegressor(
    n_estimators=550,
    min_samples_split=5,
    min_samples_leaf=5,
    max_features='sqrt',
    max_depth=8,
    random_state=42,
    n_jobs=-1
)

# Entrenar con tus datos
best_rf.fit(X_train, y_train, sample_weight=weights)
best_pred = best_rf.predict(X_val)

# RMSE para RUL<50 (críticos)
mask_critica = y_val < 50
rmse_criticos = np.sqrt(mean_squared_error(y_val[mask_critica], best_pred[mask_critica]))

# Resultados completos
print("📊 MEJOR MODELO - MÉTRICAS COMPLETAS:")
print(f"MAE total:     {mean_absolute_error(y_val, best_pred):.2f}")
print(f"MAE RUL<50:    {mean_absolute_error(y_val[mask_critica], best_pred[mask_critica]):.2f}")
print(f"RMSE total:    {np.sqrt(mean_squared_error(y_val, best_pred)):.2f}")
print(f"RMSE RUL<50:   {rmse_criticos:.2f}")  
print(f"R² total:      {r2_score(y_val, best_pred):.4f}")
print(f"Nº críticos:   {mask_critica.sum():,} ({mask_critica.mean()*100:.1f}%)")


📊 MEJOR MODELO - MÉTRICAS COMPLETAS:
MAE total:     35.83
MAE RUL<50:    12.15
RMSE total:    53.37
RMSE RUL<50:   14.19
R² total:      -0.3811
Nº críticos:   24,643 (62.1%)


### Pickle

In [ ]:
with open("../../data/random_forest.pkl", "wb") as f:
    pickle.dump(best_rf, f)